In [ ]:
%run QM9GNN2_Multitask.ipynb

In [ ]:
import itertools

In [ ]:
optimizer = Adam(lr=learning_rate)
loss = 'mse'

In [ ]:
def generate_single_task_filename(task):
    return path.join('single_task_trained_models', task + '.h5')

In [ ]:
def build_single_task_model(*, N, F, S):
  X_in = Input(shape=(N, F))
  A_in = Input(shape=(N, N))
  E_in = Input(shape=(N, N, S))

  gc1 = EdgeConditionedConv(64, activation='relu')([X_in, A_in, E_in])
  gc2 = EdgeConditionedConv(128, activation='relu')([gc1, A_in, E_in])
  pool = GlobalAttentionPool(256)(gc2)
  dense = Dense(256, activation='relu')(pool)
  output = Dense(1)(dense)

  # Build model
  model = Model(inputs=[X_in, A_in, E_in], outputs=output)
  optimizer = Adam(lr=learning_rate)
  model.compile(optimizer=optimizer, loss='mse')

  return model

In [ ]:
tasks = [task for cluster in clusters for task in cluster]
for task in tasks:
    print('learning', task)
    model = build_single_task_model(N=N, F=F, S=S)
    model.compile(optimizer=optimizer, loss=loss)
    # es_callback = EarlyStopping(monitor='val_loss', patience=es_patience)
    model.fit([X_train, A_train, E_train],
             y_train[[task]].values,
             batch_size=batch_size,
             validation_split=0.1,
             epochs=epochs)
    model.save_weights(generate_single_task_filename(task))

In [ ]:
def load_and_compile_model(task):
    model = build_single_task_model(N=N, F=F, S=S)
    model.load_weights(generate_single_task_filename(task))
    model.compile(optimizer=optimizer, loss=loss)

In [ ]:
def calculate_transfer_coefficient_from_file(task_learned, task_transferred, y_transferred_test):
    model_learned = build_single_task_model(N=N, F=F, S=S)
    model_transferred = build_single_task_model(N=N, F=F, S=S)
    model_learned.load_weights(generate_single_task_filename(task_learned))
    model_transferred.load_weights(generate_single_task_filename(task_transferred))
    
    layers_learned = model_learned.get_weights()
    layers_transferred = model_transferred.get_weights()    
    
    # 10 is a hard-coded value dependent on the architecture
    layers_transferred = layers_learned[:10] + layers_transferred[10:]
    model_transferred.set_weights(layers_transferred)
    model_transferred.compile(optimizer=optimizer, loss=loss)
    
    eval_results = model_transferred.evaluate([X_test, A_test, E_test],
                                              y_transferred_test,
                                              batch_size=batch_size)
    return eval_results

In [ ]:
def load_and_test_single_task_model(task_learned, y_test):
    model = build_single_task_model(N=N, F=F, S=S)
    model.load_weights(path.join('single_task_trained_models', task_learned + '.h5'))
    model.compile(optimizer=optimizer, loss=loss)
    eval_results = model.evaluate([X_test, A_test, E_test], y_test, batch_size=batch_size)
    return eval_results

In [ ]:
print(load_and_test_single_task_model('A', y_test[['A']].values))
print(calculate_transfer_coefficient_from_file('A', 'B', y_test[['B']].values))

In [ ]:
print(tasks)
transfer_coefficient_dict = dict()
for task_learned, task_transferred in itertools.permutations(tasks, 2):
    print(task_learned, task_transferred)
    transfer_coefficient = calculate_transfer_coefficient_from_file(task_learned, task_transferred, y_test[[task_transferred]].values)
    print(transfer_coefficient)
    
    if task_learned not in transfer_coefficient_dict.keys():
        transfer_coefficient_dict[task_learned] = {task_transferred: transfer_coefficient}
    else:
        transfer_coefficient_dict[task_learned][task_transferred] = transfer_coefficient

In [ ]:
file = open('cross_task_transfer_coefficients.txt', 'w')
for key1, value in transfer_coefficient_dict.items():
    for key2, coef in value.items():
        print(key1, key2, coef)
        print(key1, key2, coef, file=file)

In [ ]:
# The rest of this file is comprised of mostly useless analysis code.

In [ ]:
def analyze_transfer_coefficients():
    with open('cross_task_transfer_coefficients.txt', 'r') as file:
        lines = file.readlines()
        transfer_coefficient_dict = dict()
        for line in lines:
            task_learned, task_transferred, transfer_coefficient = line.strip().split()
            if task_learned not in transfer_coefficient_dict.keys():
                transfer_coefficient_dict[task_learned] = {task_transferred: float(transfer_coefficient)}
            else:
                transfer_coefficient_dict[task_learned][task_transferred] = float(transfer_coefficient)
        return transfer_coefficient_dict

In [ ]:
transfer_coefficient = analyze_transfer_coefficients()
coef_list = list()
# relevant_tasks = ['alpha', 'homo', 'lumo', 'gap']
relevant_tasks = ['mu', 'homo']
for task_set in itertools.combinations(relevant_tasks, 3):
    coef_sum = 0
    for i, j in itertools.combinations(task_set, 2):
        coef_sum += transfer_coefficient[i][j]**2 + transfer_coefficient[j][i]**2
    coef_list.append((coef_sum, task_set))
# coef_list = np.asarray(coef_list)
# coef_list = np.sort(coef_list)
coef_list.sort()
for entry in coef_list:
    print(entry)

In [ ]:
for i, j in itertools.combinations(transfer_coefficient.keys(), 2):
    if transfer_coefficient[i][j] < 0.9 and transfer_coefficient[j][i] < 0.9:
        print(i, 
              j, 
              round(transfer_coefficient[i][j], 3), 
              round(transfer_coefficient[j][i], 3))